In [ ]:
!pip install keras_vggface

In [ ]:
%tensorflow_version 1.x

In [ ]:
import os
import numpy as np
import tensorflow as tf
import keras
from keras.engine import  Model
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Dropout, Flatten, GlobalAveragePooling2D
from keras import optimizers
from keras_vggface.vggface import VGGFace
from keras.callbacks import ReduceLROnPlateau

In [ ]:
print(tf.__version__)
print(keras.__version__)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Data Preparation

In [ ]:
# data are in dataset folder in zipped format
!ls "drive/My Drive/ENGR635-Deep Learning System Design Project/Dataset/fer2013/"

In [ ]:
!ls ../content/

In [ ]:
# This creates a temporary folder in drive root folder, so it will have to be reloaded again when required after terminating the session
# Permanent data are stored in dataset in zipped format
# This copies the zipped file and store in root of google drive temporarily
! rm -rf Training; mkdir Training
! unzip -q "drive/My Drive/ENGR635-Deep Learning System Design Project/Dataset/fer2013/Training.zip" -d Training

! rm -rf Validation; mkdir Validation
! unzip -q "drive/My Drive/ENGR635-Deep Learning System Design Project/Dataset/fer2013/PublicTest.zip" -d Validation

! rm -rf Test; mkdir Test
! unzip -q "drive/My Drive/ENGR635-Deep Learning System Design Project/Dataset/fer2013/PrivateTest.zip" -d Test

In [ ]:
!ls

In [ ]:
!ls -l Training/

In [ ]:
%%bash
root='Training/'
IFS=$(echo -en "\n\b")
(for dir in $(ls -1 "$root")
    do printf "$dir: " && ls -i "$root$dir" | wc -l
 done)

In [ ]:
%%bash
root='Validation/'
IFS=$(echo -en "\n\b")
(for dir in $(ls -1 "$root")
    do printf "$dir: " && ls -i "$root$dir" | wc -l
 done)

In [ ]:
%%bash
root='Test/'
IFS=$(echo -en "\n\b")
(for dir in $(ls -1 "$root")
    do printf "$dir: " && ls -i "$root$dir" | wc -l
 done)

In [ ]:
train_dir = "Training/"
validation_dir = "Validation/"
test_dir = "Test/"

## All data are ready

In [ ]:
# Setting image height and width
img_height = 224
img_width = 224

In [ ]:
# Image Data Generator setup
train_datagen = ImageDataGenerator(rescale=1./255,
                                   featurewise_center=False,
                                   featurewise_std_normalization=False,
                                   rotation_range=30,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   zoom_range=0.1,
                                   horizontal_flip=True
                                   )
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (img_height,img_width),
    batch_size = 50,
    shuffle = True,
    class_mode='categorical'
)
validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size = (img_height,img_width),
    batch_size = 50,
    class_mode = 'categorical'
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size = (img_height,img_width),
    batch_size = 50,
    class_mode = 'categorical'
)

In [ ]:
print(train_generator.class_indices)
print(validation_generator.class_indices)
print(test_generator.class_indices)

In [ ]:
def class_weight_computer():
  """
    Training Data categories and number of samples in them
    Angry: 3995
    Disgust: 436
    Fear: 4097
    Happy: 7215
    Neutral: 4965
    Sad: 4830
    Surprise: 3171
  """
  samples_per_label = [3995, 436, 4097, 7215, 4965, 4830, 3171]
  total_samples = sum(samples_per_label)
  return dict([(i, total_samples/(7*j)) for (i,j) in enumerate(samples_per_label)]) # https://scikit-learn.org/stable/modules/generated/sklearn.utils.class_weight.compute_class_weight.html
class_weights = class_weight_computer()

### Loading the pretrained model

In [ ]:
# https://github.com/rcmalli/keras-vggface#projects--blog-posts
conv_base = VGGFace(model='vgg16',
                  include_top = False,
                  input_shape = (img_height, img_width, 3))

In [ ]:
conv_base.summary()

In [ ]:
print("Number of trainable weights before freezing the conv base:", len(conv_base.trainable_weights))
conv_base.trainable = False
print("Number of trainable weights after freezing the conv base:", len(conv_base.trainable_weights))

In [ ]:
conv_base.summary()

### Adding Attention network + Classifier on top of pretrained network

In [ ]:
from keras.layers import Reshape, GlobalMaxPooling2D, GlobalAveragePooling2D, Add, multiply, Activation, Lambda, Concatenate
from keras import backend as K
from keras.activations import sigmoid
DROP_OUT_RATE = 0.5
channel_axis = 3
ratio = 8
kernel_size = 3

input_feature = conv_base.output
channel = input_feature._keras_shape[channel_axis]

shared_layer_one = Dense(channel//ratio,
                         activation='relu',
                         kernel_initializer='he_normal',
                         use_bias=True,
                         bias_initializer='zeros')
shared_layer_two = Dense(channel,
                         kernel_initializer='he_normal',
                         use_bias=True,
                         bias_initializer='zeros')
# For channel attention
# Average pooling
avg_pool = GlobalAveragePooling2D()(input_feature)
# print(avg_pool)
avg_pool = Reshape((1,1,channel))(avg_pool)
# print(avg_pool)
avg_pool = shared_layer_one(avg_pool)
avg_pool = shared_layer_two(avg_pool)

# Max Pooling
max_pool = GlobalMaxPooling2D()(input_feature)
max_pool = Reshape((1,1,channel))(max_pool)
max_pool = shared_layer_one(max_pool)
# print(max_pool)
max_pool = shared_layer_two(max_pool)
# print(max_pool)

cbam_feature = Add()([avg_pool,max_pool])
cbam_feature = Activation('sigmoid')(cbam_feature)
channel_attention = multiply([input_feature, cbam_feature])
# print(channel_attention)

# For spatial attention
cbam_feature = channel_attention
avg_pool = Lambda(lambda x: K.mean(x, axis=3, keepdims=True))(cbam_feature)
# print(avg_pool)

max_pool = Lambda(lambda x: K.max(x, axis=3, keepdims=True))(cbam_feature)
# print(max_pool)

concat = Concatenate(axis=3)([avg_pool, max_pool])
# print(concat)
cbam_feature = Conv2D(filters = 1,
                      kernel_size=kernel_size,
                      strides=1,
                      padding='same',
                      activation='sigmoid',
                      kernel_initializer='he_normal',
                      use_bias=False)(concat)

# print(cbam_feature)
# spatial_attention = multiply([input_feature, cbam_feature])
spatial_attention = multiply([channel_attention, cbam_feature])
# print(spatial_attention)

# print(input_feature)
cbam = Add()([spatial_attention,input_feature]) # Actual CBAM working, adding residual at end
# print(cbam)
# x = Flatten(name='flatten')(last_layer)
x = GlobalAveragePooling2D(name="global_average_pool_1")(cbam) # Global Average pooling after CBAM
# x = GlobalMaxPooling2D(name="global_max_pool_1")(cbam) # Global max pooling after CBAM

x = Dropout(DROP_OUT_RATE)(x)
x = Dense(512, activation='relu', name='fc1')(x)
x = Dropout(DROP_OUT_RATE)(x)
x = Dense(512, activation='relu', name='fc2')(x)
out = Dense(7, activation='softmax', name='classifier')(x)

model = Model(conv_base.input, out)

In [ ]:
model.summary()

In [ ]:
set_trainable = False
for layer in model.layers:
  if layer.name == "pool5":
    set_trainable = True
  if set_trainable:
    layer.trainable = True
  else:
    layer.trainable = False

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=0.0001)
model.compile(optimizer=optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
EPOCHS = 50
training_samples = 28709
validation_samples = 3589
test_samples = 3589
history = model.fit_generator(
    train_generator,
    steps_per_epoch=training_samples//batch_size,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=validation_samples//batch_size,
    shuffle=True,
    class_weight=class_weights,
    callbacks=[reduce_lr]
)

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
loss = history.history['loss']

val_acc = history.history['val_accuracy']
val_loss = history.history['val_loss']

epochs = range(1, len(acc)+1)

plt.plot(epochs, acc, 'r', label="Training accuracy")
plt.plot(epochs, val_acc, 'b', label="Validation accuracy")
plt.title('Training and Validation Accuracy with CBAM attention')
plt.legend()

plt.figure()
plt.plot(epochs, loss, 'r', label="Training Loss")
plt.plot(epochs, val_loss, 'b', label="Validation Loss")
plt.title('Training and Validation Loss with CBAM attention')
plt.legend()
plt.show()

In [ ]:
print('\nEvaluate on Validation data')
results_validation = model.evaluate_generator(validation_generator, 3589//50 )
print('Validation loss, Validation Accuracy:', results_validation)

In [ ]:
epoch_str = '-EPOCHS_' + str(EPOCHS)
val_acc = 'val_acc_%.3f' % results_validation[1]

In [ ]:
# cbam attention
print(epoch_str)
print(val_acc)

In [ ]:
# Saving the model
model.save("drive/My Drive/ENGR635-Deep Learning System Design Project/Models/VGG16_VGGFACE_Attention/" + 'VGG16_VGG_FACE_Attention' + epoch_str + val_acc + '.h5')

# Fine Tuning the trained model

### Only last layer of VGG16

Image generators and data preparation codes are pre-loaded before the below session

In [ ]:
from keras.models import load_model
model = load_model("drive/My Drive/ENGR635-Deep Learning System Design Project/Models/VGG16_VGGFACE_Attention/VGG16_VGG_FACE_Attention-EPOCHS_50_val_acc_0.457.h5")

In [ ]:
model.summary()

In [ ]:
model.trainable = True
model.summary()

In [ ]:
# Choosing last convolutional blocks of VGG16 to be trainable for fine tuning
set_trainable = False
for layer in model.layers:
  if layer.name == "conv5_3":
    set_trainable = True
  if set_trainable:
    layer.trainable = True
  else:
    layer.trainable = False

In [ ]:
# Learning rate reduced compared to original training learning rate
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.00001)
model.compile(optimizer=optimizers.Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
EPOCHS = 50
training_samples = 28709
validation_samples = 3589
test_samples = 3589
history = model.fit_generator(
    train_generator,
    steps_per_epoch=training_samples//batch_size,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=validation_samples//batch_size,
    shuffle=True,
    class_weight=class_weights,
    callbacks=[reduce_lr]
)

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
loss = history.history['loss']

val_acc = history.history['val_accuracy']
val_loss = history.history['val_loss']

epochs = range(1, len(acc)+1)

plt.plot(epochs, acc, 'r', label="Training accuracy")
plt.plot(epochs, val_acc, 'b', label="Validation accuracy")
plt.title('Training and Validation Accuracy after fine tuning 1 with cbam attention')
plt.legend()

plt.figure()
plt.plot(epochs, loss, 'r', label="Training Loss")
plt.plot(epochs, val_loss, 'b', label="Validation Loss")
plt.title('Training and Validation Loss after fine tuning 1 with cbam attention')
plt.legend()
plt.show()

In [ ]:
# 50 epochs
print('\nEvaluate on validation data')
results_validation = model.evaluate_generator(validation_generator, 3589//50)
print('val loss, val acc:', results_validation)

In [ ]:
epoch_str = '-EPOCHS_' + str(EPOCHS)
val_acc = 'test_acc_%.3f' % results_validation[1]
print(epoch_str)
print(val_acc)

In [ ]:
# Saving the model
model.save("drive/My Drive/ENGR635-Deep Learning System Design Project/Models/VGG16_VGGFACE_Attention/" + 'VGG16_VGG_FACE_attention_finetuned' + epoch_str + val_acc + '.h5')

## Finetuning 2

### Finetuning two last layer of VGG16

Image generators and data preparation codes are pre-loaded before the below session

In [ ]:
from keras.models import load_model
model = load_model("drive/My Drive/ENGR635-Deep Learning System Design Project/Models/VGG16_VGGFACE_Attention/VGG16_VGG_FACE_attention_finetuned-EPOCHS_50_val_acc_0.688.h5")

In [ ]:
model.summary()

In [ ]:
model.trainable = True
model.summary()

In [ ]:
# Choosing last convolutional blocks of VGG16 to be trainable for fine tuning
set_trainable = False
for layer in model.layers:
  if layer.name == "conv5_2":
    set_trainable = True
  if set_trainable:
    layer.trainable = True
  else:
    layer.trainable = False

In [ ]:
# Learning rate reduced compared to original training learning rate
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.000001)
model.compile(optimizer=optimizers.Adam(learning_rate=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
EPOCHS = 50
training_samples = 28709
validation_samples = 3589
test_samples = 3589
history = model.fit_generator(
    train_generator,
    steps_per_epoch=training_samples//batch_size,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=validation_samples//batch_size,
    shuffle=True,
    class_weight=class_weights,
    callbacks=[reduce_lr]
)

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
loss = history.history['loss']

val_acc = history.history['val_accuracy']
val_loss = history.history['val_loss']

epochs = range(1, len(acc)+1)

plt.plot(epochs, acc, 'r', label="Training accuracy")
plt.plot(epochs, val_acc, 'b', label="Validation accuracy")
plt.title('Training and Validation Accuracy after fine tuning 2 with cbam attention')
plt.legend()

plt.figure()
plt.plot(epochs, loss, 'r', label="Training Loss")
plt.plot(epochs, val_loss, 'b', label="Validation Loss")
plt.title('Training and Validation Loss after fine tuning 2 with cbam attention')
plt.legend()
plt.show()

In [ ]:
# 50 epochs
print('\nEvaluate on validation data')
results_validation = model.evaluate_generator(validation_generator, 3589//50)
print('val loss, val acc:', results_validation)

In [ ]:
epoch_str = '-EPOCHS_' + str(EPOCHS)
val_acc = 'val_acc_%.3f' % results_validation[1]
print(epoch_str)
print(val_acc)

In [ ]:
# Saving the model
model.save("drive/My Drive/ENGR635-Deep Learning System Design Project/Models/VGG16_VGGFACE_Attention/" + 'VGG16_VGG_FACE_attention_finetuned' + epoch_str + val_acc + '.h5')

## Finetuning 3

### Choosing last 3 layers of VGG16

Image generators and data preparation codes are pre-loaded before the below session

In [ ]:
from keras.models import load_model
model = load_model("drive/My Drive/ENGR635-Deep Learning System Design Project/Models/VGG16_VGGFACE_Attention/VGG16_VGG_FACE_attention_finetuned-EPOCHS_50_val_acc_0.701.h5")

In [ ]:
model.summary()

In [ ]:
model.trainable = True
model.summary()

In [ ]:
# Choosing last convolutional blocks of VGG16 to be trainable for fine tuning
set_trainable = False
for layer in model.layers:
  if layer.name == "conv5_1":
    set_trainable = True
  if set_trainable:
    layer.trainable = True
  else:
    layer.trainable = False

In [ ]:
# Learning rate reduced compared to original training learning rate
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.000001)
model.compile(optimizer=optimizers.Adam(learning_rate=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
EPOCHS = 50
training_samples = 28709
validation_samples = 3589
test_samples = 3589
history = model.fit_generator(
    train_generator,
    steps_per_epoch=training_samples//batch_size,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=validation_samples//batch_size,
    shuffle=True,
    class_weight=class_weights,
    callbacks=[reduce_lr]
)

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
loss = history.history['loss']

val_acc = history.history['val_accuracy']
val_loss = history.history['val_loss']

epochs = range(1, len(acc)+1)

plt.plot(epochs, acc, 'r', label="Training accuracy")
plt.plot(epochs, val_acc, 'b', label="Validation accuracy")
plt.title('Training and Validation Accuracy after fine tuning 3 with cbam attention')
plt.legend()

plt.figure()
plt.plot(epochs, loss, 'r', label="Training Loss")
plt.plot(epochs, val_loss, 'b', label="Validation Loss")
plt.title('Training and Validation Loss after fine tuning 3 with cbam attention')
plt.legend()
plt.show()

In [ ]:
# 50 epochs
print('\nEvaluate on validation data')
results_validation = model.evaluate_generator(validation_generator, 3589//50)
print('val loss, val acc:', results_validation)

In [ ]:
epoch_str = '-EPOCHS_' + str(EPOCHS)
val_acc = 'val_acc_%.3f' % results_validation[1]
print(epoch_str)
print(val_acc)

In [ ]:
# Saving the model
model.save("drive/My Drive/ENGR635-Deep Learning System Design Project/Models/VGG16_VGGFACE_Attention/" + 'VGG16_VGG_FACE_attention_finetuned' + epoch_str + val_acc + '.h5')

In [ ]:
print('\nEvaluate on Validation data')
results_validation = model.evaluate_generator(validation_generator, 3589//50 )
print('Validation loss, Validation Accuracy:', results_validation)

In [ ]:
print('\nEvaluate on test data')
results_test = model.evaluate_generator(test_generator, 3589//50)
print('test loss, test acc:', results_test)